In [1]:
# load dependencies and prepare cache

import re
import requests
import requests_cache
# from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from io import BytesIO

requests_cache.install_cache()

In [2]:
r = requests.get('https://www.irs.gov/pub/irs-soi/stateoutflow1516.csv')

pd_options = {
#     'sheet_name': f'State {direction.capitalize()}flow',
#     'header': None,
#     'dtype': 'object'
}

with BytesIO(r.content) as fh:
    df = pd.read_csv(fh, **pd_options)
    
out_df = df[(df.y1_statefips == 17) & (~df.y2_state_name.str.contains('IL'))]
out_df = out_df[['y2_statefips', 'y2_state', 'y2_state_name', 'n1']]
out_df.rename(columns={'y2_statefips': 'fips', 'y2_state': 'state', 'y2_state_name': 'state_name', 'n1': 'outflow'}, inplace=True)
out_df.reset_index(drop=True, inplace=True)
out_df

,fips,state,state_name,outflow
0,12,FL,Florida,12568
1,6,CA,California,12212
2,18,IN,Indiana,11011
3,48,TX,Texas,10925
4,55,WI,Wisconsin,9573
5,29,MO,Missouri,7813
6,4,AZ,Arizona,5699
7,26,MI,Michigan,5302
8,8,CO,Colorado,5082
9,19,IA,Iowa,4906


In [3]:
r = requests.get('https://www.irs.gov/pub/irs-soi/stateinflow1516.csv')

pd_options = {
#     'sheet_name': f'State {direction.capitalize()}flow',
#     'header': None,
#     'dtype': 'object'
}

with BytesIO(r.content) as fh:
    df = pd.read_csv(fh, **pd_options)

in_df = df[(df.y2_statefips == 17) & (~df.y1_state_name.str.contains('IL'))]
in_df = in_df[['y1_statefips', 'y1_state', 'y1_state_name', 'n1']]
in_df.rename(columns={'y1_statefips': 'fips', 'y1_state': 'state', 'y1_state_name': 'state_name', 'n1': 'inflow'}, inplace=True)
in_df.reset_index(drop=True, inplace=True)
in_df

,fips,state,state_name,inflow
0,18,IN,Indiana,7801
1,6,CA,California,7334
2,48,TX,Texas,6941
3,55,WI,Wisconsin,6666
4,29,MO,Missouri,6521
5,12,FL,Florida,6110
6,26,MI,Michigan,4603
7,19,IA,Iowa,4125
8,36,NY,New York,4067
9,39,OH,Ohio,3457


In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

SHEET = '1DHcOoRlw1co3s3it6Wkl9tXtk3EJITrwpMovrvkVOT8'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pjudge/.credentials/BGA Graphics-3edf4552f3a5.json', scope)

gc = gspread.authorize(credentials)

worksheet = gc.open_by_key(SHEET).worksheet('data')

def blank_out_worksheet(worksheet):
    """
    totally blank out worksheet
    """
    from gspread_dataframe import get_as_dataframe, set_with_dataframe
    
    zeroed_df = get_as_dataframe(worksheet)
    
    # set vals to null
    zeroed_df[:] = np.nan
    
    # set cols to null
    zeroed_df.rename(columns=lambda x: np.nan, inplace=True)
    
    # set worksheet to blank dataframe
    set_with_dataframe(worksheet, zeroed_df)
    
df_out = out_df.merge(in_df, on=['fips', 'state', 'state_name'])
df_out.fips = df_out.fips.astype(str).str.zfill(2)

blank_out_worksheet(worksheet)
set_with_dataframe(worksheet, df_out)